In [1]:
import os
from os.path import abspath, join
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
import cv2
from PIL import Image

In [2]:
# Set working directory
os.chdir("..")#print(os.chdir())
os.getcwd()

'C:\\Users\\darkg\\PycharmProjects\\cin'

In [3]:
# Read images and masks paths
images_folder = abspath("data/cin_data_preprocessed/")
masks_folder = abspath("data/cin_segmentation_masks/")
out_folder = abspath("./out")

# Read images and masks
images = os.listdir(images_folder)
masks = os.listdir(masks_folder)#io.imread()

In [9]:
def get_single_cell_coordinates(mask, cellid, expansion=10):
    # Focus on one cell at a time
    singlecell = mask == cellid

    # Get non-zero coordinates for x and y
    x,y = np.nonzero(singlecell)

    # Carefully expand the coordinates, we can not have coordinates less than 0
    # and we can not have coordinates greater than the size of the image
    x1 = x.min()-expansion if x.min()-expansion > 0 else 0
    x2 = x.max()+expansion if x.max()+expansion < mask.shape[0] else mask.shape[0]
    y1 = y.min()-expansion if y.min()-expansion > 0 else 0
    y2 = y.max()+expansion if y.max()+expansion < mask.shape[1] else mask.shape[1]

    # Return coordinates for the cell with expansion
    return x1, x2, y1, y2

def pad_image(im, size):
    # Create empy array of the desired size
    black = np.zeros(size)

    # Check if im size is smaller than desired size
    # TODO: Handle the case when is not
    if sc.shape[0] < black.shape[0] and sc.shape[1] < black.shape[1]:

        # Calculate the difference between im and size
        dif_x = black.shape[0]-sc.shape[0]
        dif_y = black.shape[1]-sc.shape[1]

        # Divide the difference between im and size by 2
        # Fill blank image with original image on the new coordinates.
        black[dif_x//2:dif_x//2+sc.shape[0], dif_y//2:dif_y//2+sc.shape[1]] = sc

    return black


def im_min_max_scaler(im, minmaxrange=(0, 1)):
    # Unpack value range
    min, max = minmaxrange

    # Normal scaling (0 - 1)
    im = (im - im.min()) / (im.max() - im.min())

    # In case of different range re-scale
    im = im * (max - min) + min

    return im


# Iterate over each file on masks path
for im_f, mask_f in zip(images, masks):
    # Running one example only
    try:
        # Read image and mask
        im = io.imread(join(images_folder, im_f))[:,:,1]
        mask = np.load(join(masks_folder, mask_f))

        # Iterate over each cell in the mask and get single cell bounding boxes sc_bb
        mask_sc_bb = [get_single_cell_coordinates(mask, i, expansion=100) for i in range(1, mask.max()+1)]


        # Iterate over each nuclei
        for cellid, coord in enumerate(mask_sc_bb):
            x1, x2, y1, y2  = coord

            # Get single cell
            sc = im[x1:x2, y1:y2]

            if sc.shape[0]/sc.shape[1] > 1.4 or sc.shape[0]/sc.shape[1] < 0.55:
                continue
            else:
                # Scale image
                sc = im_min_max_scaler(sc, minmaxrange=(0,255))

                # Pad image
                sc = pad_image(sc, (512, 512))

                # Save SC image
                cv2.imwrite(join(out_folder,f"{mask_f}_{cellid}.png"), sc)
    except IndexError:
        print(im_f, mask_f)